In [1]:
%matplotlib inline
import importlib
from pytorch_utils_oh_2 import *

Pytorch utils oh: pytorch_utils_oh_2.py
Pytorch: 0.2.0_4


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
MODEL_SAVE_PATH = 'numbers_gen_4_mod_data'

In [ ]:
import pytorch_utils_oh_2; importlib.reload(pytorch_utils_oh_2); from pytorch_utils_oh_2 import *;

# Data loading

In [4]:
all_data = pickle.load(open("data/en_train_fixed_1.pkl", "rb" ))
all_data_sentence_index = all_data.set_index('sentence_id')

In [5]:
all_data.sample(2)

,sentence_id,token_id,class,before,after,class_org
8375412,633983,1,NOT_CHANGED,on,on,PLAIN
2633878,204173,4,NOT_CHANGED,Ё,Ё,VERBATIM


In [6]:
categories_all = all_data["class"].unique()
print(categories_all)
print(len(categories_all))
categories_index = dict((c, i) for i, c in enumerate(categories_all))

['NOT_CHANGED' 'NUMBERS' 'LETTERS' 'PLAIN' 'VERBATIM' 'ELECTRONIC']
6


### Utils stuff

In [7]:
chars_normal, chars_normal_index = load_characters_pkl('data/en_features/chars_normal.pkl')
print(''.join(chars_normal))

<SOS><EOS>☒ !"#$%&'(),-./0123456789:;ABCDEFGHIJKLMNOPQRSTUVWXYZ_abcdefghijklmnopqrstuvwxyz~£¥ª²³µº¼½¾éɒʻˈΩμ—€⅓⅔⅛


In [8]:
common_words, common_words_index = load_common_words_10k()
len(common_words)
common_words[0:10]

8192

['<EOS>', '<SOS>', '<UNK>', '<0000>', '<SAMPLE>', '.', ',', 'the', '"', 'of']

### More balanced sample

In [10]:
number_data = all_data[all_data['class'] == 'NUMBERS']
print("Data rows: {},  (dropped rows: {})".format(len(number_data), len(all_data)-len(number_data)))
number_data = number_data.reset_index(drop=True)

Data rows: 448172,  (dropped rows: 9470020)


In [14]:
balanced_data = number_data

balanced_data_length = len(balanced_data)
def balanced_data_sample_row():
    global balanced_data_last_sample
    balanced_data_last_sample = balanced_data.iloc[random.randint(1, balanced_data_length-1)]
    return balanced_data_last_sample

In [15]:
 balanced_data_sample_row()

sentence_id           63001
token_id                  5
class               NUMBERS
before                 39th
after          thirty ninth
class_org           ORDINAL
Name: 38710, dtype: object

### Number words

In [17]:
arr = list(set(list(number_data['after'])))
arr = [s.split(' ') for s in arr]
arr = np.concatenate(arr)
arr = list(set(arr))
number_words = [EOS_TOKEN, SOS_TOKEN, UNKNOWN_WORD_TOKEN, NUMBER_WORD_TOKEN, SAMPLE_WORD_TOKEN] + arr
number_words_index = dict((c, i) for i, c in enumerate(number_words))
len(number_words)

511

In [2]:
number_words

NameError: name 'number_words' is not defined

In [18]:
def number_words_to_tensor(words, include_eos=True):
    return words_to_tensor(words, words_lookup_index=number_words_index, include_eos=include_eos)
number_words_to_tensor(['one', 'first']).shape

torch.Size([1, 3, 511])

In [50]:
number_words_index['first']

423

In [98]:
pickle.dump(number_words, open('data/models/numbers_gen_4_mod_data_1/number_words.pkl', 'wb'))

In [19]:
number_words_onehot_sos = number_words_to_tensor([SOS_TOKEN], include_eos=False)
#number_words_onehot_sos = Variable(torch.from_numpy(number_words_onehot_sos)).cuda()
number_words_onehot_sos.size()

torch.Size([1, 1, 511])

### Samples

In [20]:
def get_random_sample():
    sample_row = balanced_data_sample_row()
    sentence_id = sample_row['class']

    rows = all_data_sentence_index.loc[sample_row['sentence_id']]
    befores = list(rows.before)
        
    token_id_idx = list(rows['token_id']).index(sample_row['token_id'])
    befores[token_id_idx] = SAMPLE_WORD_TOKEN
    
    return sample_row['before'], sample_row['after'], sample_row['class'], befores
            
def tmp():
    s_bef, s_aft, s_class, s_sentence = get_random_sample()
    print(s_class, ':', s_bef, '->', s_aft)
    print(' '.join(s_sentence))
    print(s_sentence)
    print(words_to_tensor(list(s_sentence), common_words_index).shape)
    print(string_to_tensor(s_bef, chars_normal_index).shape)
    print(number_words_to_tensor(s_aft.split(' ')).shape)
tmp()

NUMBERS : 978-0-8112-1878-8 -> nine seven eight sil o sil eight one one two sil one eight seven eight sil eight
Katherine Silver ; 2010 ) ISBN <SAMPLE> ( New Directions ) The Seamstress and the Wind ( trans .
['Katherine', 'Silver', ';', '2010', ')', 'ISBN', '<SAMPLE>', '(', 'New', 'Directions', ')', 'The', 'Seamstress', 'and', 'the', 'Wind', '(', 'trans', '.']
torch.Size([1, 20, 8192])
torch.Size([1, 18, 104])
torch.Size([1, 18, 511])


# Model functions

In [ ]:
use_cuda = True

### Encoder

In [28]:
class EncoderRNN(nn.Module):
    def __init__(self, words_input_size, chars_input_size, words_hidden_size, chars_hidden_size,
                 words_layers=1, chars_layers=1):
        super(EncoderRNN, self).__init__()
        
        self.words_layers = words_layers
        self.chars_layers = chars_layers
        self.words_hidden_size = words_hidden_size
        self.chars_hidden_size = chars_hidden_size

        self.rnn_words = nn.LSTM(words_input_size, words_hidden_size // 2, words_layers,
                                 batch_first=True, bidirectional=True)

        self.rnn_chars = nn.LSTM(chars_input_size, chars_hidden_size // 2, chars_layers,
                                batch_first=True, bidirectional=True)
        
    def forward(self, word_vectors, string_tensor, hidden = None, init_hidden = True):
        if init_hidden:
            hidden_words, hidden_chars = self.init_hidden()
        
        all_outputs_words, hidden_words = self.rnn_words(word_vectors, hidden_words)
        output_words = all_outputs_words[:, -1]
        
        all_outputs_chars, hidden_chars = self.rnn_chars(string_tensor, hidden_chars)
        output_chars = all_outputs_chars[:, -1]
        
        output = torch.cat((output_words, output_chars), 1)
        
        return output

    def init_hidden(self):
        var1_1 = Variable(torch.zeros(2 * self.words_layers, 1, self.words_hidden_size // 2))
        var1_2 = Variable(torch.zeros(2 * self.words_layers, 1, self.words_hidden_size // 2))
        var2_1 = Variable(torch.zeros(2 * self.chars_layers, 1, self.chars_hidden_size // 2))
        var2_2 = Variable(torch.zeros(2 * self.chars_layers, 1, self.chars_hidden_size // 2))
        
        var1_1 = var1_1.cuda(); var1_2 = var1_2.cuda()
        var2_1 = var2_1.cuda(); var2_2 = var2_2.cuda()
        return ((var1_1, var1_2), (var2_1, var2_2))

In [74]:
encoder_rnn = EncoderRNN(words_input_size=len(common_words), chars_input_size=len(chars_normal),
                         words_hidden_size=128, chars_hidden_size=128,
                         words_layers=2, chars_layers=2).cuda()
encoder_rnn

EncoderRNN (
  (rnn_words): LSTM(8192, 64, num_layers=2, batch_first=True, bidirectional=True)
  (rnn_chars): LSTM(104, 64, num_layers=2, batch_first=True, bidirectional=True)
)

In [31]:
def test_encoder_single_sample():
    s_bef, s_aft, s_class, s_sentence = get_random_sample()
    
    words_t = words_to_tensor(list(s_sentence), common_words_index)
    words_t = Variable(words_t).cuda()
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    return encoder_rnn(words_t, string_t)
    
encoder_output = test_encoder_single_sample()
encoder_output.size()

torch.Size([1, 256])

### Decoder

In [76]:
class DecoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        
        self.rnn = nn.GRU(input_size, hidden_size, n_layers,
                                 batch_first=True, bidirectional=False)
                         # LSTM would require own hidden included
        
        self.lin_out = nn.Linear(hidden_size, input_size)
        #self.softmax = nn.LogSoftmax()

    def forward(self, char, hidden):
        #char = char.view(1,1,-1)
        #hidden = hidden.view(1,1,-1)
        output, hidden = self.rnn(char, hidden)
        output = output[:, -1] # view(1,-1)
        output = self.lin_out(output)
        output = F.log_softmax(output)
        return output, hidden

decoder_rnn = DecoderRNN(input_size=len(number_words), hidden_size=encoder_output.size()[-1], n_layers=1)
decoder_rnn = decoder_rnn.cuda()
decoder_rnn

DecoderRNN (
  (rnn): GRU(511, 256, batch_first=True)
  (lin_out): Linear (256 -> 511)
)

In [46]:
tmp_a, tmp_b = decoder_rnn(Variable(number_words_onehot_sos).cuda(), encoder_output.view(1,1,-1))
print(tmp_a.size())
print(tmp_a.topk(1)[1])
print(number_words[tmp_a.topk(1)[1].data[0][0]])

torch.Size([1, 511])
Variable containing:
 53
[torch.cuda.LongTensor of size 1x1 (GPU 0)]

steph


# Training etc

### Accuracy

In [56]:
def test_model_single_sample(model=None):
    s_bef, s_aft, s_class, s_sentence = sample = get_random_sample()
        
    words_t = words_to_tensor(list(s_sentence), common_words_index)
    words_t = Variable(words_t).cuda()
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    encoder_output = encoder_rnn(words_t, string_t)
    
    encoder_output = encoder_output.view(1,1,-1)
    
    decoder_hidden = encoder_output
    decoder_input = Variable(number_words_onehot_sos).cuda()

    decoded_output = []
    max_length = 20
    for _ in range(max_length):
        decoder_output, decoder_hidden = decoder_rnn(decoder_input, decoder_hidden)
        #return decoder_output

        topv, topi = decoder_output.data.topk(1)
        word_index = topi[0][0]
        word = number_words[word_index] # Use own prediction as next input
                
        if word == EOS_TOKEN:
            break

        decoded_output.append(word)
        
        decoder_input = number_words_to_tensor([word], include_eos=False)
        decoder_input = Variable(decoder_input).cuda()
    
    output = ' '.join(decoded_output)
    return output, output, s_aft, sample
    
tmp = test_model_single_sample(None)
tmp

('steph and and and and and and and and and and and and and and and and and and and',
 'steph and and and and and and and and and and and and and and and and and and and',
 'one thousand milligrams',
 ('1000 mg',
  'one thousand milligrams',
  'NUMBERS',
  ['Dosage',
   'of',
   '<SAMPLE>',
   '/day',
   'will',
   'produce',
   'a',
   '25%',
   'decrease',
   'in',
   'performance',
   ',',
   'on',
   'top',
   'of',
   'the',
   'reduction',
   'due',
   'to',
   'high',
   'altitude',
   'exposure',
   '.']))

In [58]:
def print_local_wrong_predictions(max_results=10):
    arr = get_some_wrong_predictions(None, test_model_single_sample, max_iterations=10000, max_results=max_results)
    for sample, predict, output in arr:
        s_bef, s_aft, s_class, s_sentence = sample
        print("{:<14} => {:<14} || {} \n{:>17} {}".format(s_bef, predict, s_aft, '', ' '.join(s_sentence), ))


In [60]:
print_local_wrong_predictions(2)

1931           => steph and and and and and and and and and and and and and and and and and and and || nineteen thirty one 
                  He was a member of the US Davis Cup teams in 1930 and <SAMPLE> but did not play any matches .
26             => steph and and and and and and and and and and and and and and and and and and and || twenty six 
                  Challenger landed at Edwards Air Force Base , California , on August 6, 1985 , at 12:45 : <SAMPLE> pm PDT .


In [61]:
%%time
test_model_accuracy(encoder_rnn, test_model_single_sample)

Accuracy: 0.00% (       0/   10000)
CPU times: user 11min 4s, sys: 7 s, total: 11min 11s
Wall time: 2min 57s


0.0

### Training

In [70]:
def train(s_bef, s_aft, s_sentence, encoder_optimizer, decoder_optimizer, loss_function,
          use_teacher_forcing, max_length=20):
    
    words_t = words_to_tensor(list(s_sentence), common_words_index)
    words_t = Variable(words_t).cuda()
    
    string_t = string_to_tensor(s_bef, chars_normal_index)
    string_t = Variable(string_t).cuda()
    
    encoder_output = encoder_rnn(words_t, string_t)
    encoder_output = encoder_output.view(1,1,-1)
    
    decoder_hidden = encoder_output
    decoder_input = Variable(number_words_onehot_sos).cuda()
    
    ###
    
    target_arr = s_aft.split(' ') + [EOS_TOKEN]
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0
    
    decoded_output = []
    for i in range(len(target_arr)):
        decoder_output, decoder_hidden = decoder_rnn(decoder_input, decoder_hidden)

        decoder_target_i = number_words_index[target_arr[i]]
        decoder_target_i = Variable(torch.LongTensor([decoder_target_i])).cuda()
        loss += loss_function(decoder_output, decoder_target_i)
        
        topv, topi = decoder_output.data.topk(1)
        word_index = topi[0][0]
        word = number_words[word_index] # Use own prediction as next input
        decoded_output.append(word)
        
        if use_teacher_forcing:
            word = target_arr[i] # replace input with right target
        else:
            # use output normally as input 
            if word == EOS_TOKEN:
                break
                
        decoder_input = number_words_to_tensor([word], include_eos=False)
        decoder_input = Variable(decoder_input).cuda()
        
    if decoded_output[-1] == EOS_TOKEN:
        decoded_output = decoded_output[:-1]
        
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return ' '.join(decoded_output), (loss.data[0] / len(target_arr))


In [71]:
def train_iterations(n_iters=100000, lr=0.001, teacher_forcing_ratio=0.5,
                     print_every=10000, plot_every=1000):

    start = time.time()
    
    decoder_rnn.train()
    encoder_rnn.train()

    current_loss = 0
    current_loss_iter = 0

    encoder_optimizer = torch.optim.Adam(encoder_rnn.parameters(), lr=lr)
    decoder_optimizer = torch.optim.Adam(decoder_rnn.parameters(), lr=lr)
    loss_function = nn.NLLLoss()
    
    for iteration in range(1, n_iters + 1):
        model_training.iterations += 1
        
        use_teacher_forcing = random.random() < teacher_forcing_ratio
        
        s_bef, s_aft, s_class, s_sentence = get_random_sample()
        
        result, loss = train(s_bef=s_bef, s_aft=s_aft, s_sentence=s_sentence,
                             encoder_optimizer=encoder_optimizer, decoder_optimizer=decoder_optimizer,
                             loss_function=nn.NLLLoss(), use_teacher_forcing=use_teacher_forcing,
                             max_length=40 )
        
        current_loss += loss
        current_loss_iter += 1

        # Print iter number, loss, name and guess
        if iteration % print_every == 0:
            teacher_forcing_str = ""
            if use_teacher_forcing:
                teacher_forcing_str = "(forcing)"
            correct = '✓' if result == s_aft else "✗: {}".format(s_aft)
            
            print("{:>6d} {:>4.0%} ({:>8}) {:>7.3f}   | {:>6.2f}: {} -> {} ({}) {}".format(
                      model_training.iterations, iteration/n_iters, time_since(start),
                      current_loss/current_loss_iter, loss,
                      s_bef, result, correct, teacher_forcing_str))

        # Add current loss avg to list of losses
        if iteration % plot_every == 0:
            model_training.losses.append(current_loss / plot_every)
            model_training.learning_rates.append(lr)
            current_loss = 0
            current_loss_iter = 0
            
        if model_training.iterations % 50000 == 0 or model_training.iterations == 10:
            model_training.save_models()
            acc = test_model_accuracy(encoder_rnn, test_model_single_sample)
            model_training.accuracy.append(acc)
    
    # test_model_accuracy(model, n_sample=10000)

In [77]:
model_training = ModelTraining(MODEL_SAVE_PATH, [encoder_rnn, decoder_rnn])

Save path: data/models/numbers_gen_4_mod_data_1


In [78]:
train_iterations(n_iters=50, print_every=9, lr=0.0001)

     9  18% (   0m 0s)   6.244   |   6.26: 2 -> percent fiftieths (✗: two) (forcing)
Saved model to data/models/numbers_gen_4_mod_data_1/10_(EncoderRNN/DecoderRNN)
Accuracy: 0.00% (       0/   10000)
    18  36% (  2m 19s)   6.239   |   6.23: 171,000 -> percent percent percent euros nine nine (✗: one hundred seventy one thousand) (forcing)
    27  54% (  2m 19s)   6.230   |   6.19: May 10, 2015 -> percent percent nine nine nine (✗: may tenth twenty fifteen) (forcing)
    36  72% (  2m 19s)   6.222   |   6.19: 2000 -> percent nine percent (✗: two thousand) 
    45  90% (  2m 20s)   6.214   |   6.19: July 9, 2008 -> percent nine percent nine percent nine (✗: july ninth two thousand eight) 


In [79]:
train_iterations(n_iters=(1000-model_training.iterations), print_every=500, lr=0.0001)

   550  53% (   0m 7s)   2.958   |   5.07: 1840s -> nineteen <EOS> (✗: eighteen forties) (forcing)


In [80]:
train_iterations(n_iters=9000, lr=0.0001, print_every=1000)

  2000  11% (  0m 14s)   2.611   |   1.38: II -> two (✓) (forcing)
  3000  22% (  0m 30s)   2.511   |   2.29: 50 -> one (✗: fifty) (forcing)
  4000  33% (  0m 44s)   2.477   |   2.23: 12 May 2009 -> the twenty of of twenty thousand <EOS> (✗: the twelfth of may two thousand nine) (forcing)
  5000  44% (  0m 59s)   2.325   |   2.40: 21 June 2008 -> the twenty of of twenty twenty <EOS> <EOS> (✗: the twenty first of june two thousand eight) (forcing)
  6000  56% (  1m 14s)   2.386   |   1.33: 2004 -> two thousand <EOS> (✗: two thousand four) (forcing)
  7000  67% (  1m 33s)   2.329   |   2.30: 809 -> two hundred <EOS> (✗: eight hundred nine) (forcing)
  8000  78% (  1m 52s)   2.186   |   1.74: 1,002 -> two thousand <EOS> (✗: one thousand two) (forcing)
  9000  89% (  2m 11s)   2.160   |   2.85: 2 June 1581 -> the twenty of of <EOS> <EOS> <EOS> (✗: the second of june fifteen eighty one) (forcing)
 10000 100% (  2m 30s)   2.046   |   1.18: 1958 -> nineteen ninety eight (✗: nineteen fifty eig

In [81]:
train_iterations(n_iters=90000, print_every=10000)

 20000  11% (  3m 21s)   1.195   |   1.96: 2010-08-07 -> the twenty of of twenty two thousand (✗: the seventh of august twenty ten) 
 30000  22% (  6m 51s)   0.728   |   0.02: 2004 -> two thousand four (✓) (forcing)
 40000  33% ( 10m 22s)   0.551   |   0.03: 20th -> twentieth (✓) 
 50000  44% ( 13m 42s)   0.401   |   0.44: 7th -> seventh (✓) (forcing)
Saved model to data/models/numbers_gen_4_mod_data_1/50000_(EncoderRNN/DecoderRNN)
Accuracy: 66.70% (    6670/   10000)
 60000  56% ( 18m 25s)   0.384   |   0.00: 10 -> ten (✓) (forcing)
 70000  67% ( 21m 54s)   0.294   |   1.44: 6,712 -> six thousand seven hundred thirty two (✗: six thousand seven hundred twelve) 
 80000  78% ( 25m 27s)   0.289   |   0.01: 4th -> fourth (✓) (forcing)
 90000  89% ( 28m 54s)   0.275   |   0.00: 1981 -> nineteen eighty one (✓) 
100000 100% (  32m 9s)   0.265   |   1.30: 2.7% -> two point seven seven (✗: two point seven percent) (forcing)
Saved model to data/models/numbers_gen_4_mod_data_1/100000_(EncoderRNN/

In [82]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.001)

110000   3% (  3m 16s)   0.217   |   1.70: 8247 -> eight hundred twenty seven (✗: eight thousand two hundred forty seven) 
120000   7% (  6m 32s)   0.241   |   0.20: 5pm -> five p m (✓) 
130000  10% (  9m 48s)   0.230   |   0.24: 120,000 -> one hundred twenty thousand (✓) 
140000  13% (  13m 4s)   0.215   |   0.00: 1971 -> nineteen seventy one (✓) (forcing)
150000  17% ( 16m 21s)   0.215   |   0.03: Oct 2011 -> october twenty eleven (✓) (forcing)
Saved model to data/models/numbers_gen_4_mod_data_1/150000_(EncoderRNN/DecoderRNN)
Accuracy: 87.55% (    8755/   10000)
160000  20% ( 20m 46s)   0.202   |   0.00: 1969 -> nineteen sixty nine (✓) (forcing)
170000  23% (  24m 4s)   0.168   |   0.01: 1994 -> nineteen ninety four (✓) (forcing)
180000  27% ( 27m 19s)   0.150   |   0.00: 2002 -> two thousand two (✓) 
190000  30% ( 30m 36s)   0.147   |   0.01: 283 -> two hundred eighty three (✓) 
200000  33% ( 33m 53s)   0.154   |   0.01: 1959 -> nineteen fifty nine (✓) (forcing)
Saved model to data/

In [83]:
print_local_wrong_predictions()

2,111.7        => two thousand one hundred seventeen point seven || two thousand one hundred eleven point seven 
                  On January 1, 1971 , the City of Saskatoon annexed the airport and surrounding lands totalling <SAMPLE> acres .
2012-09-30     => the thirtieth of september twenty ten || the thirtieth of september twenty twelve 
                  Banco Bradesco S / A . <SAMPLE> .
2015           => twenty fifteen fifteen || twenty fifteen 
                  Tim Buford , " Georgia - <SAMPLE> edition , Bradt Travel Guides " , p 128 .
7/15/08        => march twenty eighth || july fifteenth o eight 
                  Accessed on <SAMPLE> .
6:00 am        => six hundred b c m || six a m 
                  It is simulcasted in aksyon tv , as well in Radyo 5 92.3 News FM from 5:30 - <SAMPLE> .
April 1700     => april seventeen seventy || april seventeen hundred 
                  Generally inhospitable conditions thwarted the effort , and it was abandoned in <SAMPLE> .
0300095937 

In [84]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0.1, lr=0.001)

410000   3% (  3m 16s)   0.125   |   0.00: February 26, 2013 -> february twenty sixth twenty thirteen (✓) 
420000   7% (  6m 34s)   0.103   |   0.00: 6 -> six (✓) 
430000  10% (  9m 52s)   0.120   |   0.00: June 2009 -> june two thousand nine (✓) 
440000  13% (  13m 3s)   0.110   |   0.00: 100 -> one hundred (✓) 
450000  17% ( 15m 31s)   0.126   |   0.00: 2002 -> two thousand two (✓) 
Saved model to data/models/numbers_gen_4_mod_data_1/450000_(EncoderRNN/DecoderRNN)
Accuracy: 93.76% (    9376/   10000)
460000  20% ( 18m 50s)   0.105   |   0.01: 1843 -> eighteen forty three (✓) 
470000  23% ( 21m 19s)   0.118   |   0.00: 1990 -> nineteen ninety (✓) 
480000  27% ( 23m 47s)   0.116   |   0.00: 117 -> one hundred seventeen (✓) 
490000  30% ( 26m 15s)   0.099   |   0.00: 2001 -> two thousand one (✓) 
500000  33% ( 28m 42s)   0.085   |   0.00: 1 -> one (✓) 
Saved model to data/models/numbers_gen_4_mod_data_1/500000_(EncoderRNN/DecoderRNN)
Accuracy: 93.43% (    9343/   10000)
510000  37% (  3

In [85]:
print_local_wrong_predictions()

20             => two o          || twenty 
                  Brut y Tywysogion , Peniarth MS 20 , 223 ; Peniarth MS <SAMPLE> Tr , 117 .
2¾             => twenty five    || two and three quarters 
                  The girls' and younger boys' race is <SAMPLE> miles while the senior boys' is 4 miles .
9801           => nine thousand eight || nine eight o one 
                  Later , other versions were developed for the MSX turbo R , NEC PC- <SAMPLE> , NEC PC- Engine , and Sega Game Gear in 1991 - 1994 .
1939           => one thousand nine hundred thirty nine || nineteen thirty nine 
                  Summer time was used in 1917 - 1919 , 1921 and <SAMPLE> - 1967 .
39370          => three three three seven o || three nine three seven o 
                  PLoS ONE , 7 ( 7 ) : e <SAMPLE> .
1996           => one nine six   || nineteen ninety six 
                  Pollution prevention : Since <SAMPLE> , Unisys has reduced hazardous waste generation by approximately 95% .
351.165 km/h   

In [86]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0.5, lr=0.0001)

710000   3% (  2m 27s)   0.105   |   0.00: 2007 -> two thousand seven (✓) (forcing)
720000   7% (  4m 56s)   0.097   |   0.00: March 18, 2008 -> march eighteenth two thousand eight (✓) (forcing)
730000  10% (  7m 25s)   0.064   |   0.00: 22 August 2013 -> the twenty second of august twenty thirteen (✓) 
740000  13% (  9m 54s)   0.054   |   0.00: 600 -> six hundred (✓) (forcing)
750000  17% ( 12m 24s)   0.073   |   0.00: 2015 -> twenty fifteen (✓) (forcing)
Saved model to data/models/numbers_gen_4_mod_data_1/750000_(EncoderRNN/DecoderRNN)
Accuracy: 95.44% (    9544/   10000)
760000  20% ( 15m 42s)   0.093   |   1.96: 647986- -> six seven eight six six six six (✗: six four seven nine eight six) (forcing)
770000  23% ( 18m 11s)   0.078   |   0.00: 147 -> one hundred forty seven (✓) (forcing)
780000  27% ( 20m 40s)   0.060   |   0.00: December 12, 1953 -> december twelfth nineteen fifty three (✓) (forcing)
790000  30% (  23m 9s)   0.043   |   0.02: September 3 -> september third (✓) (forci

In [87]:
print_local_wrong_predictions()

0-8147-5123-7  => o sil eight one one sil sil one one seven seven sil three || o sil eight one four seven sil five one two three sil seven 
                  ISBN <SAMPLE> Patterson , John ( 2007 ) .
0-8128-1537-8  => o sil eight one one sil sil one eight eight five sil eight || o sil eight one two eight sil one five three seven sil eight 
                  Sampson , Anthony " The Sovereign State of ITT " , Stein and Day , 1973 , ISBN <SAMPLE> Smith , J. Richard .
10             => ten            || one o 
                  These are 07 BUJUMBURA 479 , 10 WINDHOEK 7 , 07 BUJUMBURA 515 , 09 STATE 15113 , 09 STOCKHOLM 194 , <SAMPLE> SANAA 5 , and 10 CARACAS 107 .
04.01PM IST    => o four sil p m || four o one p m i s t 
                  TNN May 28, 2013 , <SAMPLE> ( 2013-05-28 ) .
1-893777-00-6  => one sil eight seven seven seven sil sil seven seven six sil six || one sil eight nine three seven seven seven sil o o sil six 
                  ISBN <SAMPLE> ( series ) .
48825404       => f

In [88]:
train_iterations(n_iters=100000, print_every=10000, teacher_forcing_ratio=0, lr=0.001)

1010000  10% (  2m 27s)   0.104   |   0.00: 230 -> two hundred thirty (✓) 
1020000  20% (  4m 57s)   0.124   |   0.00: 13 January 1958 -> the thirteenth of january nineteen fifty eight (✓) 
1030000  30% (  7m 26s)   0.093   |   0.00: 754 -> seven hundred fifty four (✓) 
1040000  40% (  9m 55s)   0.106   |   0.00: 6 -> six (✓) 
1050000  50% ( 12m 25s)   0.120   |   0.00: 2008-12-08 -> the eighth of december two thousand eight (✓) 
Saved model to data/models/numbers_gen_4_mod_data_1/1050000_(EncoderRNN/DecoderRNN)
Accuracy: 94.37% (    9437/   10000)
1060000  60% ( 15m 44s)   0.099   |   0.00: 1996 -> nineteen ninety six (✓) 
1070000  70% ( 18m 13s)   0.119   |   0.00: 1st -> first (✓) 
1080000  80% ( 20m 42s)   0.070   |   0.00: 1990s -> nineteen nineties (✓) 
1090000  90% ( 23m 11s)   0.123   |   0.01: March 4, 1901 -> march fourth nineteen o one (✓) 
1100000 100% ( 25m 40s)   0.113   |   0.00: 2006 -> two thousand six (✓) 
Saved model to data/models/numbers_gen_4_mod_data_1/1100000_(E

In [89]:
print_local_wrong_predictions()

180's          => one nineties   || one eighties 
                  Neither player played at their best , both averaging below 90 despite hitting 8 <SAMPLE> between them .
1983-84 CHL    => one nine four three sil sil sil sil three sil three three || one nine eight three sil eight four sil c h l 
                  He won the <SAMPLE> Championship ( Adams Cup ) as a member of the Tulsa Oilers team coached by Tom Webster .
1065           => ten sixty six  || ten sixty five 
                  3 d <SAMPLE> - 67 , 283 Cal .
-96.14         => minus one six six two four three four || minus ninety six point one four 
                  The elevation is 735 feet ( 224 m ) and the coordinates are latitude 35.653 and longitude <SAMPLE> .
469,000        => four hundred sixty nine thousand o || four hundred sixty nine thousand 
                  When the second episode aired , the viewer count dipped to <SAMPLE> together , live and on timeshift .
83/100         => eighty three one thousand || eighty

In [90]:
train_iterations(n_iters=300000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

1110000   3% (  2m 29s)   0.078   |   0.00: 2007 -> two thousand seven (✓) 
1120000   7% (  4m 58s)   0.064   |   0.85: 305.90 -> three hundred five point five o o (✗: three hundred five point nine o) 
1130000  10% (  7m 27s)   0.081   |   0.00: 1 -> one (✓) 
1140000  13% (  9m 57s)   0.076   |   0.00: 2014 -> twenty fourteen (✓) 
1150000  17% ( 12m 25s)   0.059   |   0.00: August 1965 -> august nineteen sixty five (✓) 
Saved model to data/models/numbers_gen_4_mod_data_1/1150000_(EncoderRNN/DecoderRNN)
Accuracy: 96.29% (    9629/   10000)
1160000  20% ( 15m 43s)   0.064   |   0.00: 1986 -> nineteen eighty six (✓) 
1170000  23% ( 18m 11s)   0.070   |   0.00: 100 -> one hundred (✓) 
1180000  27% ( 20m 39s)   0.079   |   0.00: 2012 -> twenty twelve (✓) 
1190000  30% (  23m 8s)   0.056   |   0.01: 5,100 -> five thousand one hundred (✓) 
1200000  33% ( 25m 37s)   0.091   |   0.00: August 1863 -> august eighteen sixty three (✓) 
Saved model to data/models/numbers_gen_4_mod_data_1/1200000_(En

In [91]:
print_local_wrong_predictions()

61             => sixty one      || six one 
                  S- <SAMPLE> L Non amphibious civil transport version .
1,251,160      => one million two hundred one thousand thousand hundred sixty sixty || one million two hundred fifty one thousand one hundred sixty 
                  He was elected to the Parliament representing Tehran with 729,965 ( 58.3% ) out of <SAMPLE> votes .
III            => three          || the third 
                  Charles <SAMPLE> recovered these possessions in the Treaty of Paris ( 1763 ) , but ceded Florida to the British .
Saturday, 5 March 1904 => fifth the fifteenth of august nineteen o four || saturday the fifth of march nineteen o four 
                  The four Third Round matches were scheduled for <SAMPLE> .
1111           => eleven eleven  || one one one one 
                  Oregon State Capitol , R HMC- <SAMPLE> ( Salem Public Library ) Corning , Howard M. Dictionary of Oregon History .
472            => four hundred seventy two || four se

In [92]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

1410000   5% (  2m 36s)   0.037   |   0.00: August 3, 2013 -> august third twenty thirteen (✓) 
1420000  10% (  5m 11s)   0.050   |   0.54: $671,382 -> six hundred eighty three thousand one hundred seventy two dollars (✗: six hundred seventy one thousand three hundred eighty two dollars) 
1430000  15% (  7m 46s)   0.028   |   0.00: 1988 -> nineteen eighty eight (✓) 
1440000  20% ( 10m 32s)   0.067   |   0.00: 1656 -> sixteen fifty six (✓) 
1450000  25% ( 13m 10s)   0.056   |   0.00: 1874 -> eighteen seventy four (✓) 
Saved model to data/models/numbers_gen_4_mod_data_1/1450000_(EncoderRNN/DecoderRNN)
Accuracy: 96.43% (    9643/   10000)
1460000  30% ( 16m 36s)   0.051   |   0.00: 1999 -> nineteen ninety nine (✓) 
1470000  35% (  20m 1s)   0.056   |   0.00: 2013 -> twenty thirteen (✓) 
1480000  40% ( 23m 40s)   0.071   |   0.00: 1883 -> eighteen eighty three (✓) 
1490000  45% (  27m 4s)   0.057   |   0.00: 6 -> six (✓) 
1500000  50% ( 30m 43s)   0.054   |   0.00: 69.6% -> sixty nine poin

In [93]:
print_local_wrong_predictions()

1945           => one thousand nine hundred forty five || nineteen forty five 
                  Marxism in Britain : dissent , decline and re emergence <SAMPLE> - c .2000 .
9781139917148  => nine trillion seven hundred eighty one billion hundred hundred one million one hundred one one one one one one one || nine trillion seven hundred eighty one billion one hundred thirty nine million nine hundred seventeen thousand one hundred forty eight 
                  ISBN 1139917145 , <SAMPLE> .
3-8331-1253-0  => three sil eight three three one sil one three three three sil o || three sil eight three three one sil one two five three sil o 
                  ISBN <SAMPLE> " Taxon : Mandevilla laxa ( Ruiz & Pav .
V's            => the            || the fifth's 
                  The story is set in 1911 London at the time of George <SAMPLE> coronation .
£6.829m        => six point two nine nine pounds || six point eight two nine million pounds 
                  Frank is the 29th largest PR cons

In [94]:
train_iterations(n_iters=200000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

1610000   5% (  3m 28s)   0.041   |   0.00: 1966 -> nineteen sixty six (✓) 
1620000  10% (  6m 59s)   0.051   |   0.33: 132502 -> one hundred thirty two thousand five hundred (✗: one hundred thirty two thousand five hundred two) 
1630000  15% ( 10m 32s)   0.050   |   0.00: 31 December 2010 -> the thirty first of december twenty ten (✓) 
1640000  20% (  14m 6s)   0.058   |   0.00: 1 -> one (✓) 
1650000  25% ( 17m 43s)   0.050   |   0.00: 1999 -> nineteen ninety nine (✓) 
Saved model to data/models/numbers_gen_4_mod_data_1/1650000_(EncoderRNN/DecoderRNN)
Accuracy: 96.69% (    9669/   10000)
1660000  30% ( 22m 41s)   0.046   |   0.00: October 7, 2013 -> october seventh twenty thirteen (✓) 
1670000  35% ( 26m 17s)   0.053   |   0.00: October 30, 2005 -> october thirtieth two thousand five (✓) 
1680000  40% ( 29m 58s)   0.066   |   0.00: 2 -> two (✓) 
1690000  45% ( 33m 40s)   0.055   |   0.00: 2001 -> two thousand one (✓) 
1700000  50% ( 37m 11s)   0.050   |   0.00: 1586 -> fifteen eighty 

In [95]:
print_local_wrong_predictions()

II             => two second     || two 
                  Ruanui <SAMPLE> lived there with his four sons Tarauaua , Tuwhenuaroa , Koromaiterangi and Tangaroatupo .
978-3-85630-019-7 => nine seven eight sil three sil eight o three sil sil one six five sil sil seven || nine seven eight sil three sil eight five six three o sil o one nine sil seven 
                  Auflage 1985 , Daimon Verlag , ISBN <SAMPLE> Religioser Wahn und schwarze Magie , d . trag .
978-0-9723410-1-1 => nine seven eight sil o sil seven o o sil sil sil one one one sil one || nine seven eight sil o sil nine seven two three four one o sil one sil one 
                  ISBN <SAMPLE> Finley , Ruth Elbright .
Rs 535cr       => five hundred thirty five hundred || five hundred thirty five crore rupees 
                  " NHAI cancels <SAMPLE> project for 4 - lane road in Coimbatore " .
5343           => five thousand three hundred forty three || five three four three 
                  It was developed by dr Bruce Wien

In [ ]:
time.sleep(7*60*60)

In [ ]:
train_iterations(n_iters=500000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

1810000   2% (  5m 37s)   0.033   |   0.00: 1832 -> eighteen thirty two (✓) 
1820000   4% (  9m 39s)   0.059   |   0.00: 86.2% -> eighty six point two percent (✓) 
1830000   6% ( 13m 26s)   0.059   |   0.00: 2004 -> two thousand four (✓) 
1840000   8% ( 16m 49s)   0.041   |   0.00: 1993 -> nineteen ninety three (✓) 
1850000  10% ( 20m 12s)   0.053   |   0.00: July 19, 2014 -> july nineteenth twenty fourteen (✓) 
Saved model to data/models/numbers_gen_4_mod_data_1/1850000_(EncoderRNN/DecoderRNN)
Accuracy: 96.52% (    9652/   10000)
1860000  12% ( 24m 40s)   0.062   |   0.00: 2005 -> two thousand five (✓) 
1870000  14% (  28m 0s)   0.054   |   0.00: July 28 -> july twenty eighth (✓) 
1880000  16% ( 31m 21s)   0.072   |   0.00: 22 July -> the twenty second of july (✓) 
1890000  18% ( 34m 41s)   0.067   |   0.00: 124 -> one hundred twenty four (✓) 
1900000  20% ( 37m 57s)   0.044   |   0.00: 30 March 2013 -> the thirtieth of march twenty thirteen (✓) 
Saved model to data/models/numbers_gen

In [3]:
print_local_wrong_predictions()

NameError: name 'print_local_wrong_predictions' is not defined

In [ ]:
train_iterations(n_iters=500000, print_every=10000, teacher_forcing_ratio=0, lr=0.0001)

In [ ]:
print_local_wrong_predictions()